In [1]:
%cd ..

C:\Users\kubak\Desktop\engineer-project\github\LanguageModule


C:\Users\kubak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import cohere
import os
import json

# Łączenie się z api cohere

In [3]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('./api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Pobieranie dokumentów

In [4]:
documents_path =  f"./data/decoded_pdfs/"

In [5]:
def split_into_paragraphs(text):
    # Split the text into paragraphs by empty lines
    paragraphs = [paragraph.strip() for paragraph in text.split('\n\n') if paragraph.strip()]
    return paragraphs

In [6]:
# Lista do przechowywania zawartości książek
documents = []

# Iteracja przez wszystkie pliki w folderze
for i, filename in enumerate(os.listdir(documents_path)):
    if filename.endswith(".json"):  # Sprawdzenie, czy plik ma rozszerzenie .json
        file_path = os.path.join(documents_path, filename)
        
        # Otwórz plik JSON i załaduj dane
        with open(file_path, "r", encoding="utf-8") as file:
            book_content = json.load(file)

        # Dodaj każdą stronę oddzielnie
        for page in book_content.values():
            # Zamieniamy stronę na akapity, w domyśle zakładamy że najmniejszą jednostką znaczeniową jest akapit.
            paragraphs = split_into_paragraphs(page)

            for j, paragraph in enumerate(paragraphs):
                documents.append(paragraph)

        # Połącz zawartość wszystkich stron w jedną zmienną tekstową
        #full_text = " ".join(book_content.values())
        # Dodaj pełny tekst książki do listy documents
        #documents.append(full_text)

In [7]:
print(len(documents))

8732


# RAG - łączenie dokumentów

In [8]:
# Funkcja do wyszukiwania dokumentów w zbiorze
def retrieve_relevant_documents(query, documents):
    response = co.rerank(
        model='rerank-english-v3.0',
        query=query,
        documents=documents,
        top_n=5  # Liczba dokumentów do zwrócenia
    )
    return response

# Funkcja do generowania odpowiedzi
def generate_response(query, prompt):
    response = co.generate(
        model="command",  #-xlarge-nightly",  # Model generowania Cohere
        prompt=prompt,
        max_tokens=100,  # Maksymalna długość odpowiedzi
        temperature=0.7  # Stopień kreatywności odpowiedzi
    )
    return response.generations[0].text.strip()

In [9]:
# Bierzemy dokumenty z minimalną trafnością:
minimum_relevance = 0.5

# Bierzemy maksymalnie w rachubę:
maximum_documents = 3

# Przykład zapytania użytkownika
query = "I have bad squat depth."

In [10]:
# Analiza wyników z indexami
def analyze_rerank_results(response, documents):
    results = response.results  # Lista wyników
    relevant_docs = []

    for result in results:
        index = result.index  # Indeks w liście `documents`
        relevance_score = result.relevance_score  # Ocena trafności

        # Pobranie dokumentu na podstawie indeksu
        document_content = documents[index] if index is not None and index < len(documents) else None

        relevant_docs.append({
            "index": index,
            "relevance_score": relevance_score,
            "document": document_content  # Treść dokumentu
        })

    return relevant_docs

In [11]:
# Krok 1: Znalezienie najbardziej adekwatnego dokumentu
retrieval_result = retrieve_relevant_documents(query, documents)


In [12]:
relevant_docs = analyze_rerank_results(retrieval_result, documents)

# Wyświetlenie wyników
for doc in relevant_docs:
    print(f"Indeks: {doc['index']}, Trafność: {doc['relevance_score']}: \n Dokument: {doc['document']} \n")

Indeks: 6182, Trafność: 0.78282857: 
 Dokument: Squat  depth 

Indeks: 6398, Trafność: 0.46863863: 
 Dokument: what  you  do  with  the  hamstrings,  along  with  the  glutes  and  adductors.  The  drive  out  of  the 
bottom  is  hip  extension,  and  the  more  efficiently  you  use  hamstrings,  adductors,  and  glutes,  the 
more  hip drive  you  have.  This  is  another  reason  why good  depth  is  important:  the  deeper you  can 
squat  with  good  form,  the  more  the  hamstrings  are  stretched,  and  the  longer  they  are  when  they 
begin  to  contract  the  longer  they  can  produce  force  during  the  contraction. 

Indeks: 6232, Trafność: 0.38624245: 
 Dokument: A word  now about  knee  position  and  adductor, or  "groin"  muscle  function.  This  method 
utilizes  a  stance  that  requires  a  knees-out  position,  both  at  the  bottom  and  on  the  way  down. 
Knees-out  does  two  important  things  for  the  squat:  it  allows  for  easier  depth,  and  it  a

In [13]:
# Bierzemy maksymalnie w rachubę:
maximum_documents = 3
filtered_docs = [doc for doc in relevant_docs if doc["relevance_score"] > minimum_relevance]

if(len(filtered_docs) >= maximum_documents):
    filtered_docs = filtered_docs[:maximum_documents]

In [14]:
# Krok 2: Wygenerowanie odpowiedzi lub fallback
if len(filtered_docs) > 0:
    # Połaczenie wszystkich wyników
    context = " ".join([doc["document"] for doc in filtered_docs])

    # Tworzenie promptu z zapytania i znalezionego kontekstu
    prompt = f"User question: {query}\n Context: {context}\n Answer:"

    response = generate_response(query, prompt)
else:
    response = "Przepraszam, nie znalazłem odpowiedniego dokumentu, ale mogę spróbować odpowiedzieć ogólnie."

# Wyświetlenie wyniku
print("Pytanie użytkownika:", query)
print("Odpowiedź:", response)

Pytanie użytkownika: I have bad squat depth.
Odpowiedź: There are a few reasons your squat depth may be lacking. Firstly, flexibility and mobility are important factors in achieving deeper squat positions. Tight muscles, such as the hamstrings and quadriceps, can restrict your ability to fully descend into a squat. Stretching and foam rolling these muscles can help improve your flexibility and allow for a deeper squat. 

Additionally, weaknesses in certain muscles can also affect your squat depth. A weak or underactive gluteus maximus, medius, and


# Embeding model

In [15]:
# Krojenie dokumentów
documents_shortend = documents[1:100]

# Tworzenie embeddingów dokumentów
embeddings = co.embed(
    model="embed-english-v3.0",
    texts=documents_shortend,
    input_type="search_document"  
).embeddings

In [16]:
import numpy as np

# Funkcja obliczania podobieństwa kosinusowego
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Wyszukiwanie najbliższego dokumentu
def search(query, documents, embeddings):
    # Generowanie embeddingu dla zapytania
    query_embedding = co.embed(
        texts=[query],
        model="embed-english-v3.0",
        input_type="search_query"
    ).embeddings[0]
    
    # Obliczanie podobieństw
    similarities = [cosine_similarity(query_embedding, doc_emb) for doc_emb in embeddings]
    best_match_index = np.argmax(similarities)
    return documents[best_match_index], similarities[best_match_index]

In [17]:
def generate_answer(query):
    # Wyszukiwanie kontekstu
    context, similarity = search(query, documents, embeddings)

    print("context", context)
    # Generowanie odpowiedzi na podstawie kontekstu
    prompt = f"Question: {query}\n Context: {context}\n Answer:"
    response = co.generate(
        model="command-xlarge-nightly",
        prompt=prompt,
        max_tokens=200,
        temperature=0.7
    )
    return response.generations[0].text.strip()


In [18]:
# Testowanie
query = "What is low bar squat?"
answer = generate_answer(query)
print("Odpowiedź:", answer)

context Introduction
2
Odpowiedź: The low-bar squat is a weight training exercise that heavily involves the posterior chain (the lower back, glutes and hamstrings) and to a lesser degree the quadriceps.
